In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
# if using mac
df = pd.read_excel('/Users/zhanghantang/PycharmProjects/sucide_project/BIOM40forUSC.xlsx')
data = df[df['SI'].notnull()]
y = data['SI']
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'CFIS']

In [8]:
# column 'CFI-S.PheneVisit' data type is string, i am not clear its internal meaning and how to convert to float data type
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
x = x.drop(labels='CFI-S.PheneVisit', axis=1)
# drop these column directly
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'RAB3GAP2Biom240234_at']
y = np.array(y)
X = StandardScaler().fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
print(len(X_train), len(X_test))

# todo unified randome state
unified_random_state = 32
y_list = list(y)
print(y_list.count(0), y_list.count(1), y_list.count(2), y_list.count(3))

603 151
547 64 49 57


In [5]:
'''
-------------------------
the content below is about logistic regression content
-------------------------
'''

547

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='saga')
# 不同的solver可能只支持不同的有限的penalty
param_grid = {'penalty':['l1', 'l2'], 'C':[0.1, 1, 10, 100], 'max_iter':[3000, 3500], 'multi_class':['ovr', 'multinomial']}

# The macro-averaged F1 score gives equal weight to each class, best metric for our data
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro', return_train_score=True)
grid_search.fit(X, y)
print("Best penalty: ", grid_search.best_params_['penalty'])
print("Best C: ", grid_search.best_params_['C'])
print("Best score: ", grid_search.best_score_)

/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-pac

Best penalty:  l1
Best C:  10
Best score:  0.20134905232127337


/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [11]:
grid_search.best_params_

{'C': 10, 'max_iter': 3000, 'multi_class': 'multinomial', 'penalty': 'l1'}

In [15]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import make_scorer
scoring = ['accuracy', 'precision', 'recall', 'f1_macro', 'roc_auc']
scoring = {
    'accuracy': 'accuracy',
    'precision':make_scorer(precision_score, zero_division=0, average='macro'),
    'recall' :make_scorer(recall_score, average='macro'),
    'f1':'f1_macro',
    'roc_auc':'roc_auc'
}
lr_model = LR(solver='saga', penalty='l1', C=10, max_iter=3000, multi_class='multinomial')
scores = cross_validate(lr_model, X, y, cv=5, scoring=scoring)
print("Scores: accuracy is {}, mean accuary is {}".format(scores['test_accuracy'], scores['test_accuracy'].mean()))
print("Scores: precision is {}, mean precision is {}".format(scores['test_precision'], scores['test_precision'].mean()))
print("Scores: recall is {}, mean recall is {}".format(scores['test_recall'], scores['test_recall'].mean()))
print("Scores: f1 is {}, mean f1 is {}".format(scores['test_f1'], scores['test_f1_macro'].mean()))
print("Scores: roc_auc is {}, mean roc_auc is {}".format(scores['test_roc_auc'], scores['test_roc_auc'].mean()))


'''
Scores: accuracy is [0.8013245  0.76821192 0.74834437 0.76821192 0.73333333], mean accuary is 0.7638852097130242
Scores: precision is [0.44444444 0.25       0.09090909 0.23076923 0.35      ], mean precision is 0.2732245532245532
Scores: recall is [0.13793103 0.10344828 0.03448276 0.10714286 0.5       ], mean recall is 0.17660098522167486
Scores: f1 is [0.21052632 0.14634146 0.05       0.14634146 0.41176471], mean f1 is 0.19299478970021897
Scores: roc_auc is [0.5607688  0.63793103 0.58790277 0.67799071 0.7192623 ], mean roc_auc is 0.6367711207799311
'''

/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 87, in __call__
    score = scorer._score(cached_call, estimator,
  File "/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached 

Scores: accuracy is [nan nan nan nan nan], mean accuary is nan
Scores: precision is [nan nan nan nan nan], mean precision is nan
Scores: recall is [nan nan nan nan nan], mean recall is nan


/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 87, in __call__
    score = scorer._score(cached_call, estimator,
  File "/Users/zhanghantang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 328, in _score
    raise ValueError("{0} format is n

KeyError: 'test_f1_macro'